In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
%load_ext tensorboard

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import pickle

2023-05-25 06:56:46.846806: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 06:56:49.596194: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-25 06:56:49.596298: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255, # Scaling
                                   shear_range=0.2, # Data Augmentation
                                   zoom_range=0.2,
                                   validation_split=0.2 # Validation set
                                  )

In [4]:
training_data_path = "Italian Food Images"

target_size = (224,224)
batch_size = 32

train_set = datagen.flow_from_directory(
    training_data_path,
    target_size=target_size,
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical',
    subset="training",
    seed = 24)
validation_set = datagen.flow_from_directory(
    training_data_path,
    target_size=target_size,
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical',
    subset="validation",
    seed = 24)

Found 401 images belonging to 11 classes.
Found 100 images belonging to 11 classes.


In [6]:
image_shape = train_set.image_shape
print("Image Shape:", image_shape)

num_classes = train_set.num_classes
print("Number of Classes:", num_classes)

Image Shape: (224, 224, 3)
Number of Classes: 11


In [7]:
from tensorflow.keras.applications import ResNet101, InceptionResNetV2, MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_classifier_on_top(base_model, dropout_rate=0.3, num_classes = num_classes):
    inputs = base_model.input
    x = Dense(512, activation='relu')(base_model.output)
    x = Dropout(dropout_rate)(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model          

In [8]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
early_stopping = EarlyStopping(monitor='val_loss', patience=7)
tensorboard_italia = TensorBoard(log_dir='logs/mobilenet_top_italia/')

In [9]:
mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=image_shape, pooling='max')
mobilenet.trainable = False
mobilenet_top_italia = build_classifier_on_top(mobilenet)
mobilenet_top_italia.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2023-05-25 06:57:29.472915: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-25 06:57:29.485014: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-25 06:57:29.486675: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-25 06:57:29.488966: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [10]:
history_mobilenet_italia = mobilenet_top_italia.fit(train_set, validation_data=validation_set, epochs=50, batch_size=batch_size,
                          callbacks=[early_stopping, tensorboard_italia])
mobilenet_top_italia.save("MobileNetV2_italia.h5")
with open('train_history/history_mobilenet_italia', 'wb') as file:
    pickle.dump(history_mobilenet_italia.history, file)

Epoch 1/50


2023-05-25 06:57:38.904090: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8200
2023-05-25 06:57:40.690231: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x55ae27350080 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-25 06:57:40.690276: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2023-05-25 06:57:40.762470: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-25 06:57:41.392117: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


13/13 [==============================] - 39s 3s/step - loss: 3.6276 - accuracy: 0.1995 - val_loss: 1.7544 - val_accuracy: 0.4100
Epoch 2/50
13/13 [==============================] - 16s 1s/step - loss: 2.1211 - accuracy: 0.3541 - val_loss: 1.3398 - val_accuracy: 0.5700
Epoch 3/50
13/13 [==============================] - 16s 1s/step - loss: 1.3481 - accuracy: 0.5387 - val_loss: 0.8050 - val_accuracy: 0.8300
Epoch 4/50
13/13 [==============================] - 16s 1s/step - loss: 0.9828 - accuracy: 0.6833 - val_loss: 0.6041 - val_accuracy: 0.8200
Epoch 5/50
13/13 [==============================] - 16s 1s/step - loss: 0.7779 - accuracy: 0.7207 - val_loss: 0.5698 - val_accuracy: 0.8200
Epoch 6/50
13/13 [==============================] - 16s 1s/step - loss: 0.6929 - accuracy: 0.7805 - val_loss: 0.5059 - val_accuracy: 0.8600
Epoch 7/50
13/13 [==============================] - 16s 1s/step - loss: 0.6018 - accuracy: 0.7855 - val_loss: 0.4075 - val_accuracy: 0.8700
Epoch 8/50
13/13 [=============

In [11]:
mobilenet_top_italia.trainable = True
mobilenet_top_italia.compile(optimizer=keras.optimizers.Adam(1e-4),  # Very low learning rate
               loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
tensorboard_mobilenet_ft_italia = TensorBoard(log_dir='logs/mobilenet_ft_italia/')

history_mobilenet_italia_ft = mobilenet_top_italia.fit(train_set,
                                         validation_data=validation_set,
                                         epochs=100,
                                         batch_size=batch_size,
                          callbacks=[early_stopping, tensorboard_mobilenet_ft_italia])
mobilenet_top_italia.save("MobileNetV2_italia_ft.h5")
with open('train_history/history_mobilenet_ft_italia', 'wb') as file:
    pickle.dump(history_mobilenet_italia_ft.history, file)

Epoch 1/100
13/13 [==============================] - 42s 1s/step - loss: 1.3026 - accuracy: 0.6259 - val_loss: 0.3952 - val_accuracy: 0.8700
Epoch 2/100
13/13 [==============================] - 16s 1s/step - loss: 0.5942 - accuracy: 0.8279 - val_loss: 0.4673 - val_accuracy: 0.8800
Epoch 3/100
13/13 [==============================] - 16s 1s/step - loss: 0.3023 - accuracy: 0.8953 - val_loss: 0.4376 - val_accuracy: 0.8900
Epoch 4/100
13/13 [==============================] - 16s 1s/step - loss: 0.2234 - accuracy: 0.9252 - val_loss: 0.6245 - val_accuracy: 0.8500
Epoch 5/100
13/13 [==============================] - 16s 1s/step - loss: 0.1788 - accuracy: 0.9352 - val_loss: 0.6626 - val_accuracy: 0.8800
Epoch 6/100
13/13 [==============================] - 16s 1s/step - loss: 0.1156 - accuracy: 0.9651 - val_loss: 0.5469 - val_accuracy: 0.9000


NameError: name 'mobilenet_top' is not defined

In [15]:
mobilenet_top_italia.evaluate(train_set)
mobilenet_top_italia.evaluate(validation_set)

4/4 [==============================] - 4s 818ms/step - loss: 0.6029 - accuracy: 0.8600


[0.602929949760437, 0.8600000143051147]